In [1]:
# import libraries
import numpy as np
import cv2 as cv

In [2]:
# read in single test image
img = cv.imread("/home/leo/Documents/UNI/ICDS/Groupwork/Group1/data/test/normal/IM-0115-0001.jpeg", cv.IMREAD_GRAYSCALE)
print(img.shape)

(1858, 2090)


In [60]:
class conv_layer:
    def __init__(self, in_dim, conv_size=(3,3), conv_num=4):
        self.conv_size = conv_size
        self.conv_num = conv_num
        self.conv_kernels = [None] * self.conv_num
        for i in range(0, self.conv_num):
            self.conv_kernels[i] = np.empty(conv_size)
        self.in_dim = in_dim
        self.out_dim = ( self.conv_num, self.in_dim[0] - (conv_size[0] - 1), self.in_dim[1] - (self.conv_size[1] - 1))
        
    def forward(self, img):
        out_img = np.zeros(self.out_dim)
        for i in range(0, self.conv_num):
            for ii in range(0, self.out_dim[1]):
                for iii in range(0, self.out_dim[2]):
                    out_img[i, ii, iii] = np.sum(img[ii:ii+self.conv_size[0], iii:iii+self.conv_size[1]] * self.conv_kernels[i])
        return out_img
    
    def get_out_dim(self):
        return self.out_dim

In [61]:
# test conv layer
a = np.zeros((6,6))
index = 0
for i in range(0,a.shape[0]):
    for ii in range(0,a.shape[1]):
        a[i, ii] = index
        index += 1

print("input before conv")
print(a)
c = conv_layer(in_dim=a.shape, conv_size=(3,3), conv_num=3)
print("expected out dim")
print(c.get_out_dim())
d = c.forward(a)
print("output conv shape")
print(d.shape)
print("output conv")
print(d)


input before conv
[[ 0.  1.  2.  3.  4.  5.]
 [ 6.  7.  8.  9. 10. 11.]
 [12. 13. 14. 15. 16. 17.]
 [18. 19. 20. 21. 22. 23.]
 [24. 25. 26. 27. 28. 29.]
 [30. 31. 32. 33. 34. 35.]]
expected out dim
(3, 4, 4)
output conv shape
(3, 4, 4)
output conv
[[[ 42258.  46980.  51702.  56424.]
  [ 70590.  75312.  80034.  84756.]
  [ 98922. 103644. 108366. 113088.]
  [127254. 131976. 136698. 141420.]]

 [[ 42258.  46980.  51702.  56424.]
  [ 70590.  75312.  80034.  84756.]
  [ 98922. 103644. 108366. 113088.]
  [127254. 131976. 136698. 141420.]]

 [[  1320.   1482.   1644.   1806.]
  [  2292.   2454.   2616.   2778.]
  [  3264.   3426.   3588.   3750.]
  [  4236.   4398.   4560.   4722.]]]


In [55]:
class max_pooling_layer:
    def __init__(self, in_dim, pooling_size=(3,3)):
        self.pooling_size = pooling_size
        self.in_dim = in_dim
        self.out_dim = (self.in_dim[0], self.in_dim[1] - (self.pooling_size[0] - 1), self.in_dim[2] - (self.pooling_size[1] - 1))
        
    def forward(self, img):
        out_img = np.empty(self.out_dim)
        
        for i in range(0, self.out_dim[0]):
            for ii in range(0, self.out_dim[1]):
                for iii in range(0, self.out_dim[2]):
                    out_img[i,ii,iii] = np.max(img[i, ii:ii+self.pooling_size[0], iii:iii+self.pooling_size[1]])
        return out_img

In [62]:
# test pool layer
a = np.zeros((3, 5, 5))
index = 0
for i in range(0,a.shape[1]):
    for ii in range(0,a.shape[2]):
        a[:, i, ii] = index
        index += 1
print("input before pool")
print(a)
c = max_pooling_layer(in_dim=a.shape, pooling_size=(3,3))
d = c.forward(a)
print("output pool")
print(d)

input before pool
[[[ 0.  1.  2.  3.  4.]
  [ 5.  6.  7.  8.  9.]
  [10. 11. 12. 13. 14.]
  [15. 16. 17. 18. 19.]
  [20. 21. 22. 23. 24.]]

 [[ 0.  1.  2.  3.  4.]
  [ 5.  6.  7.  8.  9.]
  [10. 11. 12. 13. 14.]
  [15. 16. 17. 18. 19.]
  [20. 21. 22. 23. 24.]]

 [[ 0.  1.  2.  3.  4.]
  [ 5.  6.  7.  8.  9.]
  [10. 11. 12. 13. 14.]
  [15. 16. 17. 18. 19.]
  [20. 21. 22. 23. 24.]]]
output pool
[[[12. 13. 14.]
  [17. 18. 19.]
  [22. 23. 24.]]

 [[12. 13. 14.]
  [17. 18. 19.]
  [22. 23. 24.]]

 [[12. 13. 14.]
  [17. 18. 19.]
  [22. 23. 24.]]]


In [88]:
# currently only ReLU activation function hardcoded
class activation_layer:
    def __init__(self, in_dim):
        self.in_dim = in_dim

    def forward(self, img):
        out_img = np.stack(np.vectorize(self.relu)(img), axis=0)
        return out_img

    def relu(self, el):
        return(np.maximum(0, el))

In [93]:
# test activation layer
a = np.zeros((2, 3, 3))
index = -4
for i in range(0,a.shape[1]):
    for ii in range(0,a.shape[2]):
        a[:, i, ii] = index
        index += 1
print("input before activation")
print(a)
r = activation_layer(a.shape)
b = r.forward(a)
print(b)


input before activation
[[[-4. -3. -2.]
  [-1.  0.  1.]
  [ 2.  3.  4.]]

 [[-4. -3. -2.]
  [-1.  0.  1.]
  [ 2.  3.  4.]]]
[[[0. 0. 0.]
  [0. 0. 1.]
  [2. 3. 4.]]

 [[0. 0. 0.]
  [0. 0. 1.]
  [2. 3. 4.]]]


In [ ]:
class fully_connected_layer:
    def __init__(self, in_dim):
        pass

In [ ]:
# test them in succesion

